In [169]:
import tiktoken
from datasets import load_dataset

In [170]:
encoding = tiktoken.get_encoding("cl100k_base")

In [171]:
encoding._special_tokens

{'<|endoftext|>': 100257,
 '<|fim_prefix|>': 100258,
 '<|fim_middle|>': 100259,
 '<|fim_suffix|>': 100260,
 '<|endofprompt|>': 100276}

In [7]:
encoding.encode("test")

[1985]

In [74]:
raw_dataset = raw_dataset.map(lambda x: {"ids": encoding.encode(x["text"])})

Loading cached processed dataset at /Users/sidbaskaran/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-45ce7928739699e8.arrow


In [75]:
raw_dataset = raw_dataset.remove_columns(["text"])

In [133]:
tokens = raw_dataset.to_dict()["ids"]

In [134]:
max_sequence_length = 256

In [145]:
encoding._special_tokens["<|endofprompt|>"]

100276

In [135]:
import torch
from torch.nn import functional as F

In [136]:
for i in range(len(tokens)):
    if len(tokens[i]) > max_sequence_length:
        extra_seq = tokens[i][max_sequence_length:]
        tokens[i] = torch.tensor(tokens[i][:max_sequence_length])
        while len(extra_seq) > max_sequence_length:
            padded_tokens = extra_seq[:max_sequence_length]
            padded_tokens = F.pad(
                torch.tensor(padded_tokens),
                (0, max_sequence_length - len(padded_tokens)),
                value=-1,
            )
            tokens.append(padded_tokens)
            extra_seq = extra_seq[max_sequence_length:]
    else:
        tokens[i] = F.pad(
            torch.tensor(tokens[i]), (0, max_sequence_length - len(tokens[i])), value=-1
        )

In [141]:
data = torch.stack(tokens)

In [142]:
data.shape

torch.Size([23800, 256])

In [160]:
x = torch.tensor([[1, 1, 1, 1, 1, 0, 0, 0]])

In [161]:
x

tensor([[1, 1, 1, 1, 1, 0, 0, 0]])

In [162]:
x = x.expand(x.size(0), 8, 8)

In [167]:
torch.tril(x).squeeze()

tensor([[1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0]])